In [1]:
pwd

'D:\\Thesis Work\\InitialDemoWork'

In [2]:
import os
import tgt
import IPython.display as ipd
import librosa
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook

In [3]:
SAMPLE_RATE = 44100
MAX_NFRAME = 25578 #to make (50,13) all phones safe in 99.7%
mfccs = []
phones = []
n_frames = []
#timeSeriesList = []


files = [file[:-4] for file in os.listdir('./selectedToBeWavs/') if '.wav' in file]

for file in tqdm_notebook(files): #a file per word
    data, rate = librosa.core.load("./selectedToBeWavs/"+file+".wav", sr=SAMPLE_RATE)
    ann = tgt.io.read_textgrid('./alignedWavs/selectedToBeWavs/'+file+".TextGrid") #UTF-8
    #Tiers: words, phones, IPA(UTF-8)
    phoneTier = ann.get_tier_by_name("phones") #format: (start,end,text)

    for phone in phoneTier: #for every phone
        timeSeries = data[int(phone.start_time*rate):int(phone.end_time*rate)]
        if timeSeries.size <= MAX_NFRAME:
            timeSeries=np.concatenate((timeSeries,np.zeros(MAX_NFRAME-timeSeries.size)))
            #timeSeriesList.append(timeSeries)
            mfccs.append(librosa.feature.mfcc(timeSeries, sr = SAMPLE_RATE, n_mfcc=13))
            #mfccs.append(mfcc(timeSeries,SAMPLE_RATE,nfilt=13,nfft=2048,winfunc=np.hamming))
            n_frames.append(timeSeries.size)
            phones.append(phone.text)
            
#Some sample of the timeseries are listened to check consistency. Checks out.Very hard to listen to the phones though

In [4]:
df = pd.DataFrame({'phones' : phones, 'n_frames': n_frames, 'MFCCs': mfccs})

In [5]:
df.phones = [''.join([alpha for alpha in phone if alpha.isalpha()]) for phone in df.phones]

In [6]:
df.shape

(224722, 3)

In [7]:
df.head()

,phones,n_frames,MFCCs
0,AA,25578,"[[-212.6292397588255, -180.4339980080033, -166..."
1,sp,25578,"[[-588.6476307487844, -601.2085347365278, -621..."
2,R,25578,"[[-233.00378201793862, -234.23804420914087, -2..."
3,sp,25578,"[[-687.5199944609631, -679.0043939829035, -693..."
4,EY,25578,"[[-179.22319403266283, -143.06206693968687, -1..."


In [8]:
df.phones.nunique()

41

In [9]:
sum(df.n_frames == 25578)

224722

In [10]:
np.all(df.MFCCs.apply(np.shape) == (13,50))

True

In [11]:
for phone in df.phones.unique():
    df[df.phones==phone].drop(["phones","n_frames"],axis=1).to_pickle("./Cambridge_bigger USING LIBROSA/phoneDfs/"+phone+"_df.pickle")

In [12]:
np.save("./Cambridge_bigger USING LIBROSA/phoneList.npy",np.array(df.phones.unique()))

#### Delta Features:

In [4]:
phoneList = np.load("./Cambridge_bigger USING LIBROSA/phoneList.npy",allow_pickle=True)
for phone in tqdm_notebook(phoneList):
    df = pd.read_pickle("./Cambridge_bigger USING LIBROSA/phoneDfs/"+phone+"_df.pickle")
    print(df.shape)
    df['deltaC'] = df.MFCCs.apply(librosa.feature.delta)
    df['doubleDeltaC'] = df.MFCCs.apply(librosa.feature.delta, order=2)
    X = np.array((df.MFCCs,df.deltaC,df.doubleDeltaC))
    X = np.transpose(X,[1,2,3,0])
    np.save('./Cambridge_bigger USING LIBROSA/CAMBRIDGE_BIGGER/'+phone+'_X.npy',X) #Feature vector batch

(3889, 1)
(20712, 1)
(10364, 1)
(3786, 1)
(4367, 1)
(8690, 1)
(21038, 1)
(4927, 1)
(9416, 1)
(12562, 1)
(3862, 1)
(3568, 1)
(13505, 1)
(12399, 1)
(2691, 1)
(14294, 1)
(7424, 1)
(6341, 1)
(2828, 1)
(1839, 1)
(5788, 1)
(2738, 1)
(13094, 1)
(6155, 1)
(2661, 1)
(1478, 1)
(2400, 1)
(2154, 1)
(1649, 1)
(2180, 1)
(2213, 1)
(2486, 1)
(739, 1)
(190, 1)
(6113, 1)
(726, 1)
(1289, 1)
(1111, 1)
(556, 1)
(295, 1)
(205, 1)



In [5]:
3889+20712+10364+3786+4367+8690+21038+4927+9416+12562+3862+3568+13505+12399+2691+14294+7424+6341+2828+1839+5788+2738+13094+6155+2661+1478+2400+2154+1649+2180+2213+2486+739+190+6113+726+1289+1111+556+295+205

224722

In [6]:
X.shape

(205, 13, 50, 3)

_______

---------